In [58]:
# RUN THIS
import json
import codecs
import csv
import pandas as pd
import numpy as np

NEO_VER_CSV = 'data/out/neo_ver.csv'
NEO_OUT_JSON = 'data/input/records.json'
CROSS_CSV = 'ne4out.csv'

In [38]:
import math

# Example that iterates the records.json
result = []
TOTAL = len(jdata)
CURRENT = 0
PROGRESS = 0

for r in jdata:

    # Get name pkg and create initial list for dataframes
    name = r["name"]
    frames = []

    CURRENT += 1

    if PROGRESS < math.ceil((CURRENT/TOTAL)*100):
        PROGRESS = math.ceil((CURRENT/TOTAL)*100)
        print(f"Processing {PROGRESS}%  | {CURRENT}/{TOTAL}")

    # For each pkg(name), iterate the pkg managers 
    for pkgman in r["pkgmans"]:

        # Get df from NEO_VER for that pkg and remove rows with NaN in 'repo'
        ndf = df[df['pkg_name'] == name]
        ndf = ndf[~ndf['url'].isnull()]

        # Get df without duplicates and append to dataframes list
        frames.append(ndf[ndf['pkgman'] == pkgman].drop_duplicates('url'))

    # Concat all frames and keep only duplicated on repo field. Duplicated would be those who appear < 1
    mdf = pd.concat(frames)
    udf = mdf[mdf.duplicated(subset='url', keep=False)]
    mdf = udf.drop_duplicates()

    # Only add if merged dataframe is not empty
    if not mdf.empty:
        result.append(mdf)
        
fin = pd.concat(result)
fin.to_csv("out/neo4jout3.csv", encoding='utf-8', index=False)

Processing 13%  | 1/8
Processing 25%  | 2/8
Processing 38%  | 3/8
Processing 50%  | 4/8
Processing 63%  | 5/8
Processing 75%  | 6/8
Processing 88%  | 7/8
Processing 100%  | 8/8


In [ ]:
# RUN THIS (takes 30 sec)
neov_df = pd.read_csv('../neo4j-import/out/neo_ver.csv')
nu_df = pd.read_csv('../../samples/neo4j-cross/matching_url.csv')
old_df = pd.read_csv('out/non_unique_repo.csv')
len(nu_df) # 1669 unique pkgs


In [126]:
# RUN THIS
# Get unique package managers and how many times they appear
list_pms = []
for pms in nu_df["pkgmans"]:
    for pm in pms[1:-1].split(','):
        list_pms.append(pm)
print(len(list_pms))
q = pd.DataFrame(list_pms)
q.value_counts()

3701


3701

In [100]:
# RUN THIS (get data for \label{tbl:cross-pkgs})
# Get packages managers that appear with given package manager
def get_pkgmans(pkgman):
    pkgs = nu_df[nu_df['pkgmans'].str.contains(pkgman)]
    pmns = []
    for pms in pkgs['pkgmans']:
        for pm in pms[1:-1].split(','):
            pmns.append(pm)
    pmns = list(set(pmns))
    pmns.remove(pkgman)
    pmns.sort()
    return pmns

print(get_pkgmans('freebsd12'))
print(get_pkgmans('vcpkg'))
print(get_pkgmans('conan'))
print(get_pkgmans('netbsd9'))
print(get_pkgmans('openbsd69'))
print(get_pkgmans('cargo'))
print(get_pkgmans('homebrew'))
print(get_pkgmans('npm'))
print(get_pkgmans('chromebrew'))
print(get_pkgmans('clojars'))
print(get_pkgmans('alire'))
print(get_pkgmans('luarocks'))
print(get_pkgmans('fpm'))
print(get_pkgmans('nimble'))

# ['openbsd', 'netbsd']
# ...


['cargo', 'clojars', 'conan', 'fpm', 'luarocks', 'netbsd9', 'nimble', 'npm', 'openbsd69', 'vcpkg']
['alire', 'cargo', 'conan', 'freebsd12', 'netbsd9', 'npm', 'openbsd69']
['alire', 'cargo', 'chromebrew', 'freebsd12', 'homebrew', 'netbsd9', 'npm', 'openbsd69', 'vcpkg']
['cargo', 'conan', 'freebsd12', 'luarocks', 'npm', 'openbsd69', 'vcpkg']
['cargo', 'clojars', 'conan', 'freebsd12', 'netbsd9', 'npm', 'vcpkg']
['clojars', 'conan', 'freebsd12', 'netbsd9', 'npm', 'openbsd69', 'vcpkg']
['chromebrew', 'conan', 'npm']
['cargo', 'clojars', 'conan', 'freebsd12', 'homebrew', 'luarocks', 'netbsd9', 'openbsd69', 'vcpkg']
['conan', 'homebrew']
['cargo', 'freebsd12', 'npm', 'openbsd69']
['conan', 'vcpkg']
['freebsd12', 'netbsd9', 'npm']
['freebsd12']
['freebsd12']


In [125]:
# THIS IS MAIN SCRIPT
data_path = '../../dasea_data'
path_dict = {
    'alire': '/alire/alire_dependencies_05-17-2022.csv',
    'cargo': '/cargo/cargo_dependencies_05-17-2022.csv',
    'chromebrew': '/chromebrew/chromebrew_dependencies_05-17-2022.csv',
    'clojars': '/clojars/clojars_dependencies_05-19-2022.csv',
    'conan': '/conan/conan_dependencies_05-17-2022.csv',
    'freebsd12': '/ports/freebsd12/freebsd12_dependencies_05-17-2022.csv',
    'fpm': '/fpm/fpm_dependencies_05-17-2022.csv',
    'homebrew': '/homebrew/homebrew_dependencies_05-17-2022.csv',
    'netbsd9': '/ports/netbsd9/netbsd9_dependencies_05-17-2022.csv',
    'nimble': '/nimble/nimble_dependencies_05-17-2022.csv',
    'openbsd69': '/ports/openbsd69/openbsd69_dependencies_05-17-2022.csv',
    'vcpkg': '/vcpkg/vcpkg_dependencies_05-17-2022.csv',
    'luarocks': '/luarocks/luarocks_dependencies_05-17-2022.csv',
    'npm': '/npm/npm_dependencies_05-17-2022.csv'
}

df_dict = {}

def _df(path):
    import os
    file_path = os.path.abspath(data_path + path)
    return pd.read_csv(file_path)


# outfil = pd.read_csv('out/non_unique_repo.csv')

all_urls = nu_df['url'].unique()
res_df = pd.DataFrame(columns=('pkgnames','ecosystem', 'count_deps', 'count_total'))
counter = 0

for url in all_urls[:5]:
    counter += 1
    print(counter)
    # pkgs = neov_df[neov_df['url'] == url]['pkg_name'].unique().tolist()
    pkgnames = neov_df[neov_df['url'] == url]['pkg_name'].unique().tolist()
    # num_pkgs = neov_df[neov_df.duplicated(subset=['pkgman','url'], keep=False)]
    acc_list = []
    pmlst = []
    acc = 0
    for pm in list(set(list_pms)):
        if pm != 'npm': # dirty fix - remove when ready for real data
            path = path_dict[pm]
            ddf = pd.DataFrame()
            if path in df_dict:
                ddf = df_dict[path]
            else:
                ddf = _df(data_path + path)
                df_dict[path] = ddf
            for pkgname in pkgnames:
                targets = ddf[ddf['target_name'] == pkgname]
                if len(targets) > 1:
                    acc = acc + len(targets)
                    acc_list.append(len(ddf[ddf['target_name'] == pkgname]))
                    pmlst.append(pm)
        # else:
        #     acc_list.append(-1)
    
    entry = pd.DataFrame.from_dict({
        "pkgnames": [pkgnames],
        "ecosystem":  [list(set(pmlst))],
        "count_deps": [acc_list],
        "count_total": [acc]
    })
    res_df = pd.concat([res_df, entry], ignore_index=True)



1
2
3
4
5


In [127]:
# RUN THIS to export csv
res_df.sort_values(by=['count_total'], ascending=False).to_csv('out/res_df2.csv')